<a name="topo"></a>
# Laboratório 7

Este laboratório tem como objetivo colocar em prática os tópicos vistos nas aula 15, sobre representação de textos com word embeddings contextuais e modelos de linguagem de larga escala (LLMs)\
No livro da disciplina, você poderá encontrar mais informações sobre nos seguintes capítulos:
- Transformers: [Cap 9 - SLP Book](https://web.stanford.edu/~jurafsky/slp3/9.pdf)
- LLMs: [Cap 10 - SLP Book](https://web.stanford.edu/~jurafsky/slp3/10.pdf)

## Tópicos:
0. [Bibliotecas](#topico0)
1. [Fine-tuning GPT-2 (decoder)](#topico1)
2. [Fine-tuning T5](#topico2)
3. [TAREFA](#topico3)

<a name="topico0"></a>
##Bibliotecas

[Voltar ao topo](#topo)

In [ ]:
!pip install datasets

import pandas as pd
import torch
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


<a name="topico1"></a>
## Fine-tuning GPT-2 (decoder)

[Voltar ao topo](#topo)

In [ ]:
!pip install sentencepiece accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
# Configuração de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [ ]:
# Baixar modelo e tokenizer do GPT-2
model_name = "gpt2"  # Ou "gpt2-medium", "gpt2-large", "gpt2-xl" se preferir modelos maiores
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Carregar o dataset CNN/DailyMail
dataset = load_dataset("cnn_dailymail", "3.0.0")

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
# Usar apenas uma fração do dataset para treinamento e teste caso o treinamento esteja muito demorado
fraction = 0.01  # 1% do dataset
train_dataset = dataset["train"].shuffle(seed=42).select(range(int(len(dataset["train"]) * fraction)))
validation_dataset = dataset["validation"].shuffle(seed=42).select(range(int(len(dataset["validation"]) * fraction)))
test_dataset = dataset["test"].shuffle(seed=42).select(range(int(len(dataset["test"]) * fraction)))

print(f"Tamanho do train_dataset: {len(train_dataset)}")
print(f"Tamanho do validation_dataset: {len(validation_dataset)}")
print(f"Tamanho do test_dataset: {len(test_dataset)}")

Tamanho do train_dataset: 2871
Tamanho do validation_dataset: 133
Tamanho do test_dataset: 114


In [ ]:
print("Exemplo de artigo e resumo:")
print("Artigo:", dataset['train'][0]['article'][:300], "...")
print("Resumo:", dataset['train'][0]['highlights'])

Exemplo de artigo e resumo:
Artigo: LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappoi ...
Resumo: Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .
Young actor says he has no plans to fritter his cash away .
Radcliffe's earnings from first five Potter films have been held in trust fund .


In [ ]:
# Função de preparação dos dados
def prepare_data(examples):
    inputs = ["summarize: " + article for article in examples["article"]]
    targets = examples["highlights"]
    return {"input_text": inputs, "target_text": targets}

# Aplicar a preparação dos dados
train_dataset = train_dataset.map(prepare_data, batched=True)
validation_dataset = validation_dataset.map(prepare_data, batched=True)
test_dataset = test_dataset.map(prepare_data, batched=True)

Map:   0%|          | 0/2871 [00:00<?, ? examples/s]

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

In [ ]:
# Adicionar token de padding
tokenizer.pad_token = tokenizer.eos_token

def tokenize_data(examples):
    inputs = tokenizer(
        examples["input_text"], max_length=512, padding="max_length", truncation=True
    )
    outputs = tokenizer(
        examples["target_text"], max_length=512, padding="max_length", truncation=True
    )
    inputs["labels"] = outputs["input_ids"]

    # Substituir tokens de padding em labels por -100
    inputs["labels"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in labels]
        for labels in inputs["labels"]
    ]
    return inputs


# Tokenizar os datasets
train_dataset = train_dataset.map(tokenize_data, batched=True)
validation_dataset = validation_dataset.map(tokenize_data, batched=True)
test_dataset = test_dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/2871 [00:00<?, ? examples/s]

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

In [ ]:
# Revalidar tamanhos
print("Verificando comprimentos após tokenização:")
for i in range(3):
    print(f"Exemplo {i+1}:")
    print("Input IDs length:", len(train_dataset[i]["input_ids"]))
    print("Labels length:", len(train_dataset[i]["labels"]))

Verificando comprimentos após tokenização:
Exemplo 1:
Input IDs length: 512
Labels length: 512
Exemplo 2:
Input IDs length: 512
Labels length: 512
Exemplo 3:
Input IDs length: 512
Labels length: 512


In [ ]:
# Formatar para PyTorch
train_dataset = train_dataset.remove_columns(["article", "highlights", "id", "input_text", "target_text"])
test_dataset = test_dataset.remove_columns(["article", "highlights", "id", "input_text", "target_text"])
validation_dataset = validation_dataset.remove_columns(["article", "highlights", "id", "input_text", "target_text"])


train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
validation_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2871
})

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_gpt2",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs_gpt2",
    logging_steps=100,
    save_total_limit=3,
    load_best_model_at_end=True,
    save_strategy="epoch",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)

<ipython-input-16-91ed5ee25b03>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Iniciar o treinamento
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,7.197000,7.170979
2,6.985100,7.171536
3,6.836500,7.198680


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=2154, training_loss=7.048370828867623, metrics={'train_runtime': 1236.2229, 'train_samples_per_second': 6.967, 'train_steps_per_second': 1.742, 'total_flos': 2250508271616000.0, 'train_loss': 7.048370828867623, 'epoch': 3.0})

<a name="topico2"></a>
## Fine-tuning T5

[Voltar ao topo](#topo)

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Verifique se CUDA está disponível
device = "cuda" if torch.cuda.is_available() else "cpu"

# Também podemos usar o T5-base para melhores resultados, porém o fine-tuning irá demorar bem mais
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Carregar o dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

In [ ]:
# Usar apenas uma fração do dataset para treinamento e teste caso o treinamento esteja muito demorado
fraction = 0.01  # 1% do dataset
train_dataset = dataset["train"].shuffle(seed=42).select(range(int(len(dataset["train"]) * fraction)))
test_dataset = dataset["test"].shuffle(seed=42).select(range(int(len(dataset["test"]) * fraction)))

print(f"Tamanho do train_dataset: {len(train_dataset)}")
print(f"Tamanho do test_dataset: {len(test_dataset)}")

Tamanho do train_dataset: 2871
Tamanho do test_dataset: 114


In [ ]:
# Função de preparação dos dados
def prepare_data(examples):
    inputs = ["summarize: " + article for article in examples["article"]]
    targets = examples["highlights"]
    return {"input_text": inputs, "target_text": targets}

# Aplicar a preparação dos dados
train_dataset = train_dataset.map(prepare_data, batched=True)
test_dataset = test_dataset.map(prepare_data, batched=True)

In [ ]:
# Configurando o token de padding
tokenizer.pad_token = tokenizer.eos_token

# Função de tokenização
def tokenize_data(examples):
    inputs = tokenizer(
        examples["input_text"], max_length=512, padding="max_length", truncation=True
    )
    outputs = tokenizer(
        examples["target_text"], max_length=150, padding="max_length", truncation=True
    )
    inputs["labels"] = outputs["input_ids"]
    return inputs

# Tokenizar os datasets
train_dataset = train_dataset.map(tokenize_data, batched=True)
test_dataset = test_dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/2871 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

In [ ]:
# Remover colunas desnecessárias e configurar PyTorch
train_dataset = train_dataset.remove_columns(["article", "highlights", "id", "input_text", "target_text"])
test_dataset = test_dataset.remove_columns(["article", "highlights", "id", "input_text", "target_text"])

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_t5",
    overwrite_output_dir=True,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=50,
    save_total_limit=1,
    logging_steps=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    gradient_accumulation_steps=2,  # Gradiente acumulado para simular batches maiores
    fp16=True,
    #predict_with_generate=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

<ipython-input-24-3514e6670a00>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
50,1.113700,1.111724
100,1.049800,1.063989
150,1.099400,1.057671


TrainOutput(global_step=179, training_loss=1.1352510612104192, metrics={'train_runtime': 89.0514, 'train_samples_per_second': 32.24, 'train_steps_per_second': 2.01, 'total_flos': 387618919415808.0, 'train_loss': 1.1352510612104192, 'epoch': 0.9972144846796658})

In [ ]:
# Avaliar o modelo
metrics = trainer.evaluate()
print(metrics)

# Gerar previsões para um exemplo do dataset de teste
example = test_dataset[0]
input_ids = example["input_ids"].unsqueeze(0).to(device)
output = model.generate(input_ids, max_length=50, num_beams=2)
print("Input:", tokenizer.decode(example["input_ids"], skip_special_tokens=True))
print("Target:", tokenizer.decode(example["labels"], skip_special_tokens=True))
print("Prediction:", tokenizer.decode(output[0], skip_special_tokens=True))

{'eval_loss': 1.056578278541565, 'eval_runtime': 1.0356, 'eval_samples_per_second': 110.084, 'eval_steps_per_second': 14.485, 'epoch': 0.9972144846796658}
Input: summarize: (CNN) I see signs of a revolution everywhere. I see it in the op-ed pages of the newspapers, and on the state ballots in nearly half the country. I see it in politicians who once preferred to play it safe with this explosive issue but are now willing to stake their political futures on it. I see the revolution in the eyes of sterling scientists, previously reluctant to dip a toe into this heavily stigmatized world, who are diving in head first. I see it in the new surgeon general who cites data showing just how helpful it can be. I see a revolution in the attitudes of everyday Americans. For the first time a majority, 53%, favor its legalization, with 77% supporting it for medical purposes. Support for legalization has risen 11 points in the past few years alone. In 1969, the first time Pew asked the question about 

<a name="topico3"></a>
## TAREFA: Analise e comparação entre modelos

### Avalie os resultados do modelo T5 utilizando [ROUGE](https://medium.com/nlplanet/two-minutes-nlp-learn-the-rouge-metric-by-examples-f179cc285499)

[Voltar ao topo](#topo)

In [ ]:
!pip install rouge
import torch
from rouge import Rouge

In [ ]:
# Exemplo de uso
# Escolher a frase referência para comparar com a frase gerada pelo modelo
generated_summary = "Text summarization with Transformers is efficient for producing precise and relevant summaries."
reference_summary = "Text summarization with Transformers can be used to produce precise and relevant summaries."
# Inicializar o objeto rouge
rouge = Rouge()
# Calcular o rouge
scores = rouge.get_scores(generated_summary, reference_summary)

print("r: Recall / p: Precision / f: F1-Score\n")
print('Rouge-1: Unigramas')
print(scores[0]['rouge-1'])
print('Rouge-2: Bigramas')
print(scores[0]['rouge-2'])
print('Rouge-L: Longest sequence of common words')
print(scores[0]['rouge-l'])

In [ ]:
# [ SEU CÓDIGO AQUI ]